# Optimization Methods Lab 4
- Write simplex algorithm for linear problems.
- Write down the given problem in a matrix format.
- Solve using the simplex algorithm.
- Solve after changing the constraint function contstants to 0, 2, 0.
- Compare the solutions: minimum objective function value, optimal solution, base.

In [14]:
# Original:
# min 2x1 − 3x2 − 5x4
# −x1 + x2 − x3 − x4 ≤ 8
# 2x1 + 4x2          ≤ 10
#            x3 + x4 ≤ 3
# xi ≥ 0
# My:
# min 2x1 − 3x2 − 5x4
# −x1 + x2 − x3 − x4 ≤ 0
# 2x1 + 4x2          ≤ 2
#            x3 + x4 ≤ 0
# xi ≥ 0

# First let's turn this into a maximization problem by multiplying the objective function by -1
# max -2x1 + 3x2 + 5x4
# Next add slack variables to objective function:
# max 
# z = -2x1 + 3x2 + 5x4 - 0s1 - 0s2 - 0s3
# z + 2x1 - 3x2 - 5x4 + 0s1 + 0s2 + 0s3 = 0
# Next add slack variables to constraints:
# Original:
# max 
# z = -2x1 + 3x2 + 5x4 - 0s1 - 0s2 - 0s3
# −x1 + x2 − x3 − x4 + s1 + 0s2 + 0s3 = 8
# 2x1 + 4x2 + 0x3 + 0x4 + 0s1 + s2 + 0s3 = 10
# 0x1 + 0x2 + x3 + x4 + 0s1 + 0s2 + 1s3 = 3
# x1, x2, x3, x4, s1, s2, s3 ≥ 0

# Simplex Method
- Tight (non-basic) variables
- Loose (basic) variables
* An inequality is tight when left side exactly matches the right side
* The idea behind simplex algorithm is to loosen one inequality and tighten another (while going in the direction that maximizes), repeat until we reach the optimum.

- Introducing slack variables allows the inequalities to be turned into equations. The value of the slack variable shows if the distance that the left side is from the right side. If slack variable is zero that means that the 'inequality' is tight.
- Rewrite the equations so that the left side only contains loose variables.
* x1 = 0 - means x1 is tight, because the first constraint (x1 >= 0) is tight
* x2 = 0 - means x2 is tight, because the second constraint (x2 >= 0) is tight
* s1 = 0 - means s1 is tight, because the nth constraint (−x1 + x2 − x3 − x4 <= 0), (s1 >= 0) is tight...

- Dantzig's pivot rule - select the variable with the largest positive coeficient in the objective function (because that increases the function value the most)
    - Then tighten the variable which is closest, take the largest: var_coefiecient * b_term. (We want the largest non positive ratio)
    - Swap the new loose variable to the left.
    - Replace it in the objective function and constraints.
    - When all coeficients of the objective function are all negative, that is the optimum.

In [15]:
def print_tableau(tableau, basic_vars):
    print("Current Tableau:")
    for row in tableau:
        print(["{:.3f}".format(x) for x in row])
    print("Basic vars:", basic_vars)
    print()


def pivot(tableau, pivot_row, pivot_col):
    # Normalize the pivot row
    pivot_element = tableau[pivot_row][pivot_col]
    for j in range(len(tableau[pivot_row])):
        tableau[pivot_row][j] /= pivot_element

    # Eliminate the pivot column in other rows
    for i in range(len(tableau)):
        if i != pivot_row:
            factor = tableau[i][pivot_col]
            for j in range(len(tableau[i])):
                tableau[i][j] -= factor * tableau[pivot_row][j]

In [16]:
def simplex_linear(A: list[list[float]], B: list[float], C: list[float]):
    m = len(A) # constraints count (and slack variables count)
    n = len(A[0]) # original variables count
    # Add slack variables
    for i in range(m):
        slack = [0]*m
        slack[i] = 1
        A[i].extend(slack)
    C.extend([0]*m)

    basic_vars = list(range(n, n+m)) # loose
    non_basic_vars = list(range(n)) # tight

    # Initialise tableau (m+1) x (n+m+1), last row is objective, last column is RHS.
    tableau = [row[:] + [B[i]] for i, row in enumerate(A)]
    objective_row = [-c for c in C] + [0]
    tableau.append(objective_row)

    print_tableau(tableau, basic_vars) # Print initial tableau

    while True:
        # Identify the entering variable (column)
        objective_coeffs = tableau[m][:n + m]
        entering_col = None
        max_coeff = 0
        for j, val in enumerate(objective_coeffs):
            if val < 0 and val < max_coeff:
                max_coeff = val
                entering_col = j
        if entering_col is None: # Optimal solution reached
            break

        # Identify the leaving variable (row) using the minimum ratio test
        ratios = []
        for i in range(m):
            if tableau[i][entering_col] > 1e-12:
                ratio = tableau[i][-1] / tableau[i][entering_col]
                ratios.append((ratio, i))
        if not ratios:
            raise Exception("Linear program is unbounded.")
        leaving_row = min(ratios, key=lambda x: x[0])[1]

        pivot(tableau, leaving_row, entering_col)

        basic_vars[leaving_row] = entering_col # Update basic and non-basic variables
        print_tableau(tableau, basic_vars) # Print tableau after iteration

    solution = [0] * (n + m)
    for i, bv in enumerate(basic_vars):
        solution[bv] = tableau[i][-1]
    optimal_value = tableau[-1][-1]
    return solution[:n], optimal_value

In [17]:
# General
A = [[-1, 1, -1, -1], [2, 4, 0, 0], [0, 0, 1, 1]]
B = [8, 10, 3]
C = [2, -3, -5, 0]
C = [-i for i in C]

sol, val = simplex_linear(A, B, C)
print("Optimal solution:", sol)
print("Optimal value:", -val)

Current Tableau:
['-1.000', '1.000', '-1.000', '-1.000', '1.000', '0.000', '0.000', '8.000']
['2.000', '4.000', '0.000', '0.000', '0.000', '1.000', '0.000', '10.000']
['0.000', '0.000', '1.000', '1.000', '0.000', '0.000', '1.000', '3.000']
['2.000', '-3.000', '-5.000', '0.000', '0.000', '0.000', '0.000', '0.000']
Basic vars: [4, 5, 6]

Current Tableau:
['-1.000', '1.000', '0.000', '0.000', '1.000', '0.000', '1.000', '11.000']
['2.000', '4.000', '0.000', '0.000', '0.000', '1.000', '0.000', '10.000']
['0.000', '0.000', '1.000', '1.000', '0.000', '0.000', '1.000', '3.000']
['2.000', '-3.000', '0.000', '5.000', '0.000', '0.000', '5.000', '15.000']
Basic vars: [4, 5, 2]

Current Tableau:
['-1.500', '0.000', '0.000', '0.000', '1.000', '-0.250', '1.000', '8.500']
['0.500', '1.000', '0.000', '0.000', '0.000', '0.250', '0.000', '2.500']
['0.000', '0.000', '1.000', '1.000', '0.000', '0.000', '1.000', '3.000']
['3.500', '0.000', '0.000', '5.000', '0.000', '0.750', '5.000', '22.500']
Basic vars: [

In [20]:
# My
A = [[-1, 1, -1, -1], [2, 4, 0, 0], [0, 0, 1, 1]]
B = [2, 2, 2]
C = [2, -3, 0, -5]
C = [-i for i in C]

sol, val = simplex_linear(A, B, C)
print("Optimal solution:", sol)
print("Optimal value:", -val)

Current Tableau:
['-1.000', '1.000', '-1.000', '-1.000', '1.000', '0.000', '0.000', '2.000']
['2.000', '4.000', '0.000', '0.000', '0.000', '1.000', '0.000', '2.000']
['0.000', '0.000', '1.000', '1.000', '0.000', '0.000', '1.000', '2.000']
['2.000', '-3.000', '0.000', '-5.000', '0.000', '0.000', '0.000', '0.000']
Basic vars: [4, 5, 6]

Current Tableau:
['-1.000', '1.000', '0.000', '0.000', '1.000', '0.000', '1.000', '4.000']
['2.000', '4.000', '0.000', '0.000', '0.000', '1.000', '0.000', '2.000']
['0.000', '0.000', '1.000', '1.000', '0.000', '0.000', '1.000', '2.000']
['2.000', '-3.000', '5.000', '0.000', '0.000', '0.000', '5.000', '10.000']
Basic vars: [4, 5, 3]

Current Tableau:
['-1.500', '0.000', '0.000', '0.000', '1.000', '-0.250', '1.000', '3.500']
['0.500', '1.000', '0.000', '0.000', '0.000', '0.250', '0.000', '0.500']
['0.000', '0.000', '1.000', '1.000', '0.000', '0.000', '1.000', '2.000']
['3.500', '0.000', '5.000', '0.000', '0.000', '0.750', '5.000', '11.500']
Basic vars: [4, 